# Metabolite Beta

In [2]:
import qiime2 as q2
import biom
from biom import load_table, Table
from biom.util import biom_open
import pandas as pd
from qiime2.plugins.deicode.actions import rpca
from qiime2.plugins.emperor.actions import biplot
import numpy as np
from skbio import OrdinationResults
import plotnine as pn
%matplotlib inline

/anaconda3/envs/qiime2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/envs/qiime2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/envs/qiime2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/envs/qiime2/lib/python3.6/site-packages/tensorflow/python/framework/dty

# Impot data

In [3]:
bt = load_table('/Users/swandro/Documents/Projects/Seed_Grants/Nash/Analysis/metabolomics/Kumar_BIOM_normalized_cpm.biom')
md = pd.read_csv('/Users/swandro/Documents/Projects/Seed_Grants/Nash/data/metadata/10781_20190528-121437.txt', sep='\t', index_col=0)

met_md = pd.read_csv("metabolite_metadata.tsv", sep='\t', index_col=0)
met_md = met_md.apply(axis=0, func= lambda x: [str(y).strip() for y in x])
met_md = met_md.apply(axis=0, func= lambda x: [np.nan if y=='' else y for y in x])
met_md.index.name="feature-id"

In [3]:
func= lambda x: np.nan if x== '' else x

## Filter data

In [4]:
samples_keep = set(md.query('description in ["WT","Qa-1-/-"]').index) & set(bt.ids("sample"))
bt_f = bt.filter(ids_to_keep=samples_keep, inplace=False, axis="sample")

## Beta diversity

In [48]:
?rpca

In [5]:
qza = q2.Artifact.import_data('FeatureTable[Frequency]', bt_f)
bplt, dm = rpca(qza, n_components=5)

/anaconda3/envs/qiime2/lib/python3.6/site-packages/biom/table.py:4049: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  for r in self.matrix_data.tocsr()]
/anaconda3/envs/qiime2/lib/python3.6/site-packages/biom/table.py:4052: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  return constructor(mat, index=index, columns=columns)


In [14]:
?rpca

,0,1,2,3,4,Compound_Name
387.2895;5.8916,0.087325,-0.007308,0.026826,NaN,NaN,nan
250.1283;0.3996,0.081158,0.007054,0.041989,NaN,NaN,nan
250.1285;0.401,0.080101,0.007313,0.043094,NaN,NaN,nan
258.0959;0.4529,0.073330,0.015696,0.076519,NaN,NaN,nan
217.0683;0.3214,0.073046,0.007378,0.089955,NaN,NaN,nan
...,...,...,...,...,...,...
144.1022;0.3231,-0.073339,0.013254,0.032838,NaN,NaN,nan
303.2302;8.3437,-0.074749,-0.008765,0.008892,NaN,NaN,nan
229.1546;1.8594,-0.075520,-0.009700,0.008046,NaN,NaN,nan
152.0571;0.9092,-0.080216,-0.026022,0.033364,NaN,NaN,nan


In [34]:
%matplotlib inline

In [6]:
x = bplt.view(OrdinationResults)
x.features.merge(met_md, right_index=True, left_index=True).loc[:,[0,1,2,3,4,"Compound_Name"]].sort_values(0, ascending=False)

feature_df = x.features.copy()
feature_df.columns=["axis1","axis2","axis3","axis4","axis5"]
feature_df["tot"] = feature_df.apply(axis=1, func=lambda x: sum([abs(y) for y in x]))
feature_df = feature_df.sort_values("tot", ascending=False)
feature_df = feature_df.merge(met_md, right_index=True, left_index=True)
#feature_df.loc[["Tryptamine" in x for x in feature_df.Compound_Name]]
feature_df.sort_values("axis2").head(60)

,axis1,axis2,axis3,axis4,axis5,tot,Adduct,CAS_Number,Compound_Name,Compound_Source,INCHI,IonMode,LibMZ,LibraryQualityString,Smiles
203.1392;1.2864,0.027185,-0.082141,0.022768,-0.017522,-0.030653,0.180271,nan,nan,nan,nan,nan,nan,nan,nan,nan
203.1392;0.9451,0.013983,-0.073569,0.037093,-0.025021,-0.025173,0.174839,nan,nan,nan,nan,nan,nan,nan,nan,nan
245.1861;3.3071,0.027342,-0.071794,0.016018,0.004004,-0.036793,0.155950,nan,nan,nan,nan,nan,nan,nan,nan,nan
302.2066;3.1442,-0.013585,-0.070961,0.015195,-0.023651,-0.008553,0.131945,nan,nan,nan,nan,nan,nan,nan,nan,nan
246.1427;0.4599,-0.013228,-0.069967,0.021361,-0.012329,-0.012264,0.129149,nan,nan,nan,nan,nan,nan,nan,nan,nan
302.2061;3.1308,-0.017999,-0.069459,0.018724,-0.026626,-0.007925,0.140733,M+H,nan,Val Ala Ile,Isolated,nan,Positive-20eV,302.207,Bronze,nan
245.186;3.4637,0.023930,-0.067753,0.008253,0.038868,-0.019086,0.157890,nan,nan,nan,nan,nan,nan,nan,nan,nan
260.1596;0.6111,-0.000319,-0.067264,0.029417,-0.013693,-0.006001,0.116695,nan,nan,nan,nan,nan,nan,nan,nan,nan
203.1391;0.7726,0.000141,-0.065735,0.021145,-0.043929,-0.016282,0.147232,nan,nan,nan,nan,nan,nan,nan,nan,nan
332.2175;3.3867,-0.026081,-0.065401,0.032653,-0.020788,-0.008061,0.152985,M+H,nan,Ser Leu Leu,Isolated,nan,Positive-20eV,332.218,Bronze,nan


In [7]:
pn.options.figure_size = (5,5)

plot_dat = x.samples
plot_dat.columns=["axis1","axis2","axis3","axis4","axis5"]
plot_dat = plot_dat.merge(md, right_index=True, left_index=True)

feature_df = x.features.copy()
feature_df.columns=["axis1","axis2","axis3","axis4","axis5"]
feature_df["tot"] = feature_df.apply(axis=1, func=lambda x: sum([abs(y) for y in x]))
feature_df = feature_df.sort_values("tot", ascending=False)
feature_df = feature_df.iloc[:4,:]
name_df = feature_df.merge(met_md, right_index=True, left_index=True)
name_df["short_name"] = [x.replace("Match to ","").replace("from NIST14","").replace("Spectral ","") for x in name_df.Compound_Name]
name_df["axis1"] = [row.axis1*2.5 +.005 if row.axis1 > 0 else row.axis1*2.5 -.005 for name,row in name_df.iterrows()]
name_df["axis2"] = [row.axis2*2.5 -.01 if row.axis2 < 0 else row.axis2*2.5 +.01 for name,row in name_df.iterrows()]


p = (pn.ggplot(data = plot_dat) +
    pn.geom_point( pn.aes(x ="axis1", y="axis2", shape="strain_genotype", fill= "strain_genotype"),stroke=.5, size=3) +
    pn.geom_segment( data=feature_df, mapping=pn.aes(x=0, y=0, xend ="axis1*2.5", yend="axis2*2.5"), arrow=pn.geoms.arrow(length=.06, type="closed"), size=.7) +
    pn.geom_text( data=name_df, mapping=pn.aes(x="axis1", y="axis2", label="short_name"), size=9) +
    #pn.scale_y_continuous(limits=[-.27,.21]) + 
     pn.scale_shape_manual(name=" ", labels=["Qa-1-/-","WT"],values=["^","o"]) +
     pn.scale_fill_manual(name=" ", labels=["Qa-1-/-","WT"],values=["#B0AFAF","white"]) +
     pn.scale_x_continuous(limits=[-.24,.25]) +
     pn.guides() +
    pn.labs(x="Axis 1 ({}%)".format(round(x.proportion_explained[0]*100,2)),
            y="Axis 2 ({}%)".format(round(x.proportion_explained[1]*100,2)),
           shape='') +
     pn.theme_classic() +
     pn.theme(
         axis_text = pn.element_blank(),
         panel_background = pn.element_rect(size=1, color="black", fill=None),
         axis_ticks = pn.element_blank(),
         legend_text = pn.element_text(size=12)
     )
    )

p.save("met_biplot.pdf", dpi=300)

/anaconda3/envs/qiime2/lib/python3.6/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 5 x 5 in image.
  from_inches(height, units), units), PlotnineWarning)
/anaconda3/envs/qiime2/lib/python3.6/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: met_biplot.pdf
  warn('Filename: {}'.format(filename), PlotnineWarning)


In [5]:
#Import to qiime2
qza = q2.Artifact.import_data('FeatureTable[Frequency]', bt_f)
q2_md = q2.Metadata(md.apply(axis=0, func=lambda x:[str(y) for y in x]))
q2_tax = q2.Metadata(met_md)
#Run Deicode
bplt,dm = rpca(qza,n_components=5)
#Make biplot
bplt_viz = biplot(biplot=bplt,
                  sample_metadata=q2_md,
                  feature_metadata=q2_tax,
                 number_of_features=20).visualization

/anaconda3/envs/qiime2/lib/python3.6/site-packages/biom/table.py:4049: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  for r in self.matrix_data.tocsr()]
/anaconda3/envs/qiime2/lib/python3.6/site-packages/biom/table.py:4052: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  return constructor(mat, index=index, columns=columns)
/anaconda3/envs/qiime2/lib/python3.6/site-packages/pandas/core/sparse/frame.py:854: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pyd

In [6]:
bplt_viz

<visualization: Visualization uuid: afb607dc-eeaf-4c3d-93ec-0f9a31f6458a>